***
## Pipeline Shoddy
***

In [1]:
import numpy as np
import numpy.ma as ma
import os
import glob
import subprocess
import astropy.units as u
from astropy.io import fits
from astropy.io import ascii
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from astroquery.vizier import Vizier
from astropy.stats import sigma_clipped_stats, sigma_clip
from astropy.table import Table

import matplotlib.pyplot as plt
%matplotlib inline

os.chdir('SN data')

In [2]:
!pwd

/Users/kushagra/Documents/All study material/Hubble, Analysis!/Krittika Workshop NBs/photometry/SN data


In [3]:
def load_image(path):
    """
        Returns primaryHDU data, header and filename of FITS format image.
        Takes and Image object or a filepath as parameter.
    """
    if not os.path.isfile(path):
        raise Exception("Not a valid file path")
        
    f = fits.open(path)

    data = f[0].data       #This is the primary HDU image array
    header = f[0].header
    f.close()

    path_split = path.split('/')  # To get the image filename
    imageName = path_split[-1]
    
    #imageName = path
    
    return[data, header, imageName]

In [19]:
def query_vizier(data, header):
    w = WCS(header)

    [raImage, decImage] = w.all_pix2world(data.shape[0]/2, data.shape[1]/2, 1)  #Get the RA and Dec of the center of the image

    boxsize = 30 # arcminutes          #Set the box size to search for catalog stars

    maxmag = 18                        #Magnitude cut-offs of sources to be cross-matched against
    
    Vizier.VIZIER_SERVER = 'vizier.ast.cam.ac.uk'

    catNum = 'II/349'#This is the catalog number of PS1 in Vizier
    print('\nQuerying Vizier %s around RA %.4f, Dec %.4f with a radius of %.4f arcmin'%(catNum, raImage, decImage, boxsize))

    try:
        #You can set the filters for the individual columns (magnitude range, number of detections) inside the Vizier query
        v = Vizier(columns=['*'], column_filters={"rmag":"<%.2f"%maxmag, "Nd":">6", "e_Rmag":"<1.086/3"}, row_limit=-1)
        Q = v.query_region(SkyCoord(ra = raImage, dec = decImage, unit = (u.deg, u.deg)), radius = str(boxsize)+'m', catalog=catNum, cache=False)
        #query vizier around (ra, dec) with a radius of boxsize
        #print(Q[0])
    except:
        print('I cannnot reach the Vizier database. Is the internet working?')
        
    ps1_imCoords = w.all_world2pix(Q[0]['RAJ2000'], Q[0]['DEJ2000'], 1)

    #Another round of filtering where we reject sources close to the edges
    good_cat_stars = Q[0][np.where((ps1_imCoords[0] > 500) & (ps1_imCoords[0] < 3500) & (ps1_imCoords[1] > 500) & (ps1_imCoords[1] < 3500))]
    ###print(good_cat_stars)
    
    return good_cat_stars

In [5]:
def run_sextractor(imageName):
    configFile = 'photomCat.sex'
    catalogName = imageName+'.cat'
    paramName = 'photomCat.param'
    try:
        command = 'sex -c %s %s -CATALOG_NAME %s -PARAMETERS_NAME %s' % (configFile, imageName, catalogName, paramName)
        print('Executing command: %s' % command)
        rval = subprocess.run(command.split(), check=True)
    except subprocess.CalledProcessError as err:
        print('Could not run sextractor with exit error %s'%err)
        
    return catalogName

In [6]:
def get_table_from_ldac(filename, frame=1):
    """
    Load an astropy table from a fits_ldac by frame (Since the ldac format has column 
    info for odd tables, giving it twce as many tables as a regular fits BinTableHDU,
    match the frame of a table to its corresponding frame in the ldac file).
    
    Parameters
    ----------
    filename: str
        Name of the file to open
    frame: int
        Number of the frame in a regular fits file
    """
    from astropy.table import Table
    if frame>0:
        frame = frame*2
    tbl = Table.read(filename, hdu=frame)
    return tbl

In [7]:
#sourceTable = get_table_from_ldac(catalogName)
#cleanSources = sourceTable[(sourceTable['FLAGS']==0) & (sourceTable['FWHM_WORLD'] < 2) & (sourceTable['XWIN_IMAGE']<3500) & (sourceTable['XWIN_IMAGE']>500) &(sourceTable['YWIN_IMAGE']<3500) &(sourceTable['YWIN_IMAGE']>500)]

In [8]:
def run_psfex(imageName):
    psfConfigFile = 'psfex_conf.psfex'
    catalogName = imageName+'.cat'
    try:
        command = 'psfex -c %s %s' % (psfConfigFile, catalogName)
        print('Executing command: %s' % command)
        rval = subprocess.run(command.split(), check=True)
    except subprocess.CalledProcessError as err:
        print('Could not run psfex with exit error %s'%err)
        
    configFile = 'photomCat.sex'
    psfName = imageName + '.psf'
    psfcatalogName = imageName+'.psf.cat'
    psfparamName = 'photomPSF.param' #This is a new set of parameters to be obtained from SExtractor, including PSF-fit magnitudes
    try:
        #We are supplying SExtactor with the PSF model with the PSF_NAME option
        command = 'sex -c %s %s -CATALOG_NAME %s -PSF_NAME %s -PARAMETERS_NAME %s' % (configFile, imageName, psfcatalogName, psfName, psfparamName)
        print("Executing command: %s" % command)
        rval = subprocess.run(command.split(), check=True)
    except subprocess.CalledProcessError as err:
        print('Could not run sextractor with exit error %s'%err)
        
    return psfcatalogName

In [9]:
def cross_match(cleanPSFSources, good_cat_stars):
    psfsourceCatCoords = SkyCoord(ra=cleanPSFSources['ALPHAWIN_J2000'], dec=cleanPSFSources['DELTAWIN_J2000'], frame='icrs', unit='degree')
    ps1CatCoords = SkyCoord(ra=good_cat_stars['RAJ2000'], dec=good_cat_stars['DEJ2000'], frame='icrs', unit='degree')

    #Now cross match sources

    #Set the cross-match distance threshold to 2.1 arcsec
    photoDistThresh = 2.1
    idx_psfimage, idx_psfps1, d2d, d3d = ps1CatCoords.search_around_sky(psfsourceCatCoords, photoDistThresh*u.arcsec)
    #idx_image are indexes into sourceCatCoords for the matched sources, while idx_ps1 are indexes into ps1CatCoords for the matched sources

    num_of_matches = len(idx_psfimage)
    
    print(f'Found {num_of_matches} good matches')
    
    return idx_psfimage, idx_psfps1, num_of_matches, psfsourceCatCoords

In [10]:
#psfsourceTable = get_table_from_ldac(psfcatalogName)
#cleanPSFSources = psfsourceTable[(psfsourceTable['FLAGS']==0) & (psfsourceTable['FLAGS_MODEL']==0)  & (psfsourceTable['FWHM_WORLD'] < 2) & (psfsourceTable['XMODEL_IMAGE']<3500) & (psfsourceTable['XMODEL_IMAGE']>500) &(psfsourceTable['YMODEL_IMAGE']<3500) &(psfsourceTable['YMODEL_IMAGE']>500)]

In [25]:
def correct_for_zp(ra, dec, good_cat_stars, cleanPSFSources, psfsourceCatCoords, idx_psfps1, idx_psfimage):
    psfoffsets = ma.array(good_cat_stars['rmag'][idx_psfps1] - cleanPSFSources['MAG_POINTSOURCE'][idx_psfimage])
    #Compute sigma clipped statistics
    zero_psfmean, zero_psfmed, zero_psfstd = sigma_clipped_stats(psfoffsets)
    
    photoDistThresh = 2.1
    
    source_coords = SkyCoord(ra=[ra], dec=[dec], frame='icrs', unit='degree')
    idx_source, idx_cleanpsf_source, d2d, d3d = psfsourceCatCoords.search_around_sky(source_coords, photoDistThresh*u.arcsec)
    #try:
    print(cleanPSFSources.colnames)
    
    source_psfinstmag = cleanPSFSources[idx_cleanpsf_source]['MAG_POINTSOURCE'][0]
    source_psfinstmagerr = cleanPSFSources[idx_cleanpsf_source]['MAGERR_POINTSOURCE'][0]
#     source_psfinstmag = cleanPSFSources[idx_cleanpsf_source]['MAG_POINTSOURCE']
#     source_psfinstmagerr = cleanPSFSources[idx_cleanpsf_source]['MAGERR_POINTSOURCE']

    source_psfmag = zero_psfmed + source_psfinstmag
    source_psfmagerr = np.sqrt(source_psfinstmagerr**2 + zero_psfstd**2)
#     except:
#         source_psfmag = 0
#         source_psfmagerr = 0
        
    return [source_psfmag, source_psfmagerr]

In [35]:
def main():
    nbpath = '/Users/kushagra/Documents/All study material/Hubble, Analysis!/Krittika Workshop NBs/photometry'
    #datapath = '/Users/kushagra/Documents/All\ study\ material/Hubble,\ Analysis!/Krittika\ Workshop\ NBs/photometry/SN\ data'
    datapath = '/Users/kushagra/Documents/All study material/Hubble, Analysis!/Krittika Workshop NBs/photometry/SN data'
    
    ra = 186.55127146   ###186.55#0292
    dec = 58.31425894    ###58.31#4119
    
    #JD_and_mags = np.zeros( (len(path_list),3) )
    JD_and_mags = []
    
    path_list = []

    patterns = ['2019','r']
    for pattern in patterns:
        if pattern is None: pattern = ''   # Even if we're not given a pattern, we search in directory for fits files
        #path_list.extend(glob.glob(os.path.join(datapath, pattern+'*proc.fits')))
        path_list.extend(glob.glob(os.path.join(pattern+'*proc.fits')))
    #path_list.extend(glob.glob(os.path.join('20190227161006-319-RA.wcs.proc.fits')))
    #path_list.extend(glob.glob(os.path.join('20190307162138-863-RA.wcs.proc.fits')))
    #print(path_list)
        
    for path in path_list:
        print(f'Assessing {path}\n\n')
        data, header, imageName = load_image(path)

        good_cat_stars = query_vizier(data, header)

        catalogName = run_sextractor(imageName)

        psfcatalogName = run_psfex(imageName)

        sourceTable = get_table_from_ldac(catalogName)
        cleanSources = sourceTable[(sourceTable['FLAGS']==0) & (sourceTable['FWHM_WORLD'] < 2) & (sourceTable['XWIN_IMAGE']<3500) & (sourceTable['XWIN_IMAGE']>500) &(sourceTable['YWIN_IMAGE']<3500) &(sourceTable['YWIN_IMAGE']>500)]
        psfsourceTable = get_table_from_ldac(psfcatalogName)
        cleanPSFSources = psfsourceTable[(psfsourceTable['FLAGS']==0) & (psfsourceTable['FLAGS_MODEL']==0)  & (psfsourceTable['FWHM_WORLD'] < 2) & (psfsourceTable['XMODEL_IMAGE']<3500) & (psfsourceTable['XMODEL_IMAGE']>500) &(psfsourceTable['YMODEL_IMAGE']<3500) &(psfsourceTable['YMODEL_IMAGE']>500)]

        idx_psfimage, idx_psfps1, num_of_matches, psfsourceCatCoords = cross_match(cleanPSFSources, good_cat_stars)

        source_psfmag, source_psfmagerr = correct_for_zp(ra, dec, good_cat_stars, cleanPSFSources, psfsourceCatCoords, idx_psfps1, idx_psfimage)
        
        image_data = [header['JD'], source_psfmag, source_psfmagerr]
        
        JD_and_mags.append(image_data)
        
    return JD_and_mags

In [36]:
JD_and_mags = main()

['20190227161006-319-RA.wcs.proc.fits', '20190307162138-863-RA.wcs.proc.fits', '20190312183410-499-RA.wcs.proc.fits', '20190313145140-894-RA.wcs.proc.fits', 'r_band_20190212155954-376-RA.wcs.proc.fits']
Assessing 20190227161006-319-RA.wcs.proc.fits



Querying Vizier II/349 around RA 186.3046, Dec 58.4168 with a radius of 30.0000 arcmin
Executing command: sex -c photomCat.sex 20190227161006-319-RA.wcs.proc.fits -CATALOG_NAME 20190227161006-319-RA.wcs.proc.fits.cat -PARAMETERS_NAME photomCat.param
Executing command: psfex -c psfex_conf.psfex 20190227161006-319-RA.wcs.proc.fits.cat
Executing command: sex -c photomCat.sex 20190227161006-319-RA.wcs.proc.fits -CATALOG_NAME 20190227161006-319-RA.wcs.proc.fits.psf.cat -PSF_NAME 20190227161006-319-RA.wcs.proc.fits.psf -PARAMETERS_NAME photomPSF.param
Found 30 good matches
['VIGNET', 'X_IMAGE', 'Y_IMAGE', 'XWIN_IMAGE', 'YWIN_IMAGE', 'ERRAWIN_IMAGE', 'ERRBWIN_IMAGE', 'ALPHAWIN_J2000', 'DELTAWIN_J2000', 'FLUX_RADIUS', 'FWHM_WORLD', 'FLUX_AUTO', '

/anaconda3/envs/Astro/lib/python3.7/site-packages/numpy/core/fromnumeric.py:753: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
/anaconda3/envs/Astro/lib/python3.7/site-packages/numpy/core/fromnumeric.py:753: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
/anaconda3/envs/Astro/lib/python3.7/site-packages/numpy/core/fromnumeric.py:753: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)


Executing command: sex -c photomCat.sex 20190307162138-863-RA.wcs.proc.fits -CATALOG_NAME 20190307162138-863-RA.wcs.proc.fits.cat -PARAMETERS_NAME photomCat.param
Executing command: psfex -c psfex_conf.psfex 20190307162138-863-RA.wcs.proc.fits.cat
Executing command: sex -c photomCat.sex 20190307162138-863-RA.wcs.proc.fits -CATALOG_NAME 20190307162138-863-RA.wcs.proc.fits.psf.cat -PSF_NAME 20190307162138-863-RA.wcs.proc.fits.psf -PARAMETERS_NAME photomPSF.param
Found 231 good matches
['VIGNET', 'X_IMAGE', 'Y_IMAGE', 'XWIN_IMAGE', 'YWIN_IMAGE', 'ERRAWIN_IMAGE', 'ERRBWIN_IMAGE', 'ALPHAWIN_J2000', 'DELTAWIN_J2000', 'FLUX_RADIUS', 'FWHM_WORLD', 'FLUX_AUTO', 'FLUXERR_AUTO', 'SNR_WIN', 'ELONGATION', 'FLUX_MAX', 'MAG_AUTO', 'MAGERR_AUTO', 'FLAGS', 'BACKGROUND', 'CLASS_STAR', 'FLAGS_MODEL', 'NITER_MODEL', 'XMODEL_IMAGE', 'YMODEL_IMAGE', 'FLUX_POINTSOURCE', 'FLUXERR_POINTSOURCE', 'MAG_POINTSOURCE', 'MAGERR_POINTSOURCE', 'FLUXRATIO_POINTSOURCE', 'FLUXRATIOERR_POINTSOURCE']
Assessing 2019031218341

/anaconda3/envs/Astro/lib/python3.7/site-packages/numpy/core/fromnumeric.py:753: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
/anaconda3/envs/Astro/lib/python3.7/site-packages/numpy/core/fromnumeric.py:753: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
/anaconda3/envs/Astro/lib/python3.7/site-packages/numpy/core/fromnumeric.py:753: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
/anaconda3/envs/Astro/lib/python3.7/site-packages/numpy/core/fromnumeric.py:753: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
/anaconda3/envs/Astro/lib/python3.7/site-packages/numpy/core/fromnumeric.py:753: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedA

Executing command: sex -c photomCat.sex 20190312183410-499-RA.wcs.proc.fits -CATALOG_NAME 20190312183410-499-RA.wcs.proc.fits.cat -PARAMETERS_NAME photomCat.param
Executing command: psfex -c psfex_conf.psfex 20190312183410-499-RA.wcs.proc.fits.cat
Executing command: sex -c photomCat.sex 20190312183410-499-RA.wcs.proc.fits -CATALOG_NAME 20190312183410-499-RA.wcs.proc.fits.psf.cat -PSF_NAME 20190312183410-499-RA.wcs.proc.fits.psf -PARAMETERS_NAME photomPSF.param
Found 220 good matches
['VIGNET', 'X_IMAGE', 'Y_IMAGE', 'XWIN_IMAGE', 'YWIN_IMAGE', 'ERRAWIN_IMAGE', 'ERRBWIN_IMAGE', 'ALPHAWIN_J2000', 'DELTAWIN_J2000', 'FLUX_RADIUS', 'FWHM_WORLD', 'FLUX_AUTO', 'FLUXERR_AUTO', 'SNR_WIN', 'ELONGATION', 'FLUX_MAX', 'MAG_AUTO', 'MAGERR_AUTO', 'FLAGS', 'BACKGROUND', 'CLASS_STAR', 'FLAGS_MODEL', 'NITER_MODEL', 'XMODEL_IMAGE', 'YMODEL_IMAGE', 'FLUX_POINTSOURCE', 'FLUXERR_POINTSOURCE', 'MAG_POINTSOURCE', 'MAGERR_POINTSOURCE', 'FLUXRATIO_POINTSOURCE', 'FLUXRATIOERR_POINTSOURCE']
Assessing 2019031314514

/anaconda3/envs/Astro/lib/python3.7/site-packages/numpy/core/fromnumeric.py:753: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
/anaconda3/envs/Astro/lib/python3.7/site-packages/numpy/core/fromnumeric.py:753: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
/anaconda3/envs/Astro/lib/python3.7/site-packages/numpy/core/fromnumeric.py:753: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
/anaconda3/envs/Astro/lib/python3.7/site-packages/numpy/core/fromnumeric.py:753: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
/anaconda3/envs/Astro/lib/python3.7/site-packages/numpy/core/fromnumeric.py:753: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedA

Executing command: sex -c photomCat.sex 20190313145140-894-RA.wcs.proc.fits -CATALOG_NAME 20190313145140-894-RA.wcs.proc.fits.cat -PARAMETERS_NAME photomCat.param
Executing command: psfex -c psfex_conf.psfex 20190313145140-894-RA.wcs.proc.fits.cat
Executing command: sex -c photomCat.sex 20190313145140-894-RA.wcs.proc.fits -CATALOG_NAME 20190313145140-894-RA.wcs.proc.fits.psf.cat -PSF_NAME 20190313145140-894-RA.wcs.proc.fits.psf -PARAMETERS_NAME photomPSF.param
Found 243 good matches
['VIGNET', 'X_IMAGE', 'Y_IMAGE', 'XWIN_IMAGE', 'YWIN_IMAGE', 'ERRAWIN_IMAGE', 'ERRBWIN_IMAGE', 'ALPHAWIN_J2000', 'DELTAWIN_J2000', 'FLUX_RADIUS', 'FWHM_WORLD', 'FLUX_AUTO', 'FLUXERR_AUTO', 'SNR_WIN', 'ELONGATION', 'FLUX_MAX', 'MAG_AUTO', 'MAGERR_AUTO', 'FLAGS', 'BACKGROUND', 'CLASS_STAR', 'FLAGS_MODEL', 'NITER_MODEL', 'XMODEL_IMAGE', 'YMODEL_IMAGE', 'FLUX_POINTSOURCE', 'FLUXERR_POINTSOURCE', 'MAG_POINTSOURCE', 'MAGERR_POINTSOURCE', 'FLUXRATIO_POINTSOURCE', 'FLUXRATIOERR_POINTSOURCE']
Assessing r_band_201902

/anaconda3/envs/Astro/lib/python3.7/site-packages/numpy/core/fromnumeric.py:753: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
/anaconda3/envs/Astro/lib/python3.7/site-packages/numpy/core/fromnumeric.py:753: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
/anaconda3/envs/Astro/lib/python3.7/site-packages/numpy/core/fromnumeric.py:753: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
/anaconda3/envs/Astro/lib/python3.7/site-packages/numpy/core/fromnumeric.py:753: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
/anaconda3/envs/Astro/lib/python3.7/site-packages/numpy/core/fromnumeric.py:753: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedA

Executing command: sex -c photomCat.sex r_band_20190212155954-376-RA.wcs.proc.fits -CATALOG_NAME r_band_20190212155954-376-RA.wcs.proc.fits.cat -PARAMETERS_NAME photomCat.param
Executing command: psfex -c psfex_conf.psfex r_band_20190212155954-376-RA.wcs.proc.fits.cat
Executing command: sex -c photomCat.sex r_band_20190212155954-376-RA.wcs.proc.fits -CATALOG_NAME r_band_20190212155954-376-RA.wcs.proc.fits.psf.cat -PSF_NAME r_band_20190212155954-376-RA.wcs.proc.fits.psf -PARAMETERS_NAME photomPSF.param
Found 74 good matches
['VIGNET', 'X_IMAGE', 'Y_IMAGE', 'XWIN_IMAGE', 'YWIN_IMAGE', 'ERRAWIN_IMAGE', 'ERRBWIN_IMAGE', 'ALPHAWIN_J2000', 'DELTAWIN_J2000', 'FLUX_RADIUS', 'FWHM_WORLD', 'FLUX_AUTO', 'FLUXERR_AUTO', 'SNR_WIN', 'ELONGATION', 'FLUX_MAX', 'MAG_AUTO', 'MAGERR_AUTO', 'FLAGS', 'BACKGROUND', 'CLASS_STAR', 'FLAGS_MODEL', 'NITER_MODEL', 'XMODEL_IMAGE', 'YMODEL_IMAGE', 'FLUX_POINTSOURCE', 'FLUXERR_POINTSOURCE', 'MAG_POINTSOURCE', 'MAGERR_POINTSOURCE', 'FLUXRATIO_POINTSOURCE', 'FLUXRATIO

/anaconda3/envs/Astro/lib/python3.7/site-packages/numpy/core/fromnumeric.py:753: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
/anaconda3/envs/Astro/lib/python3.7/site-packages/numpy/core/fromnumeric.py:753: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
/anaconda3/envs/Astro/lib/python3.7/site-packages/numpy/core/fromnumeric.py:753: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)


In [37]:
JD_and_mags

[[2458542.17367477, 15.260371757507325, 0.10200960263506523],
 [2458550.18169471, 15.500420341491697, 0.23503664102626431],
 [2458555.27372752, 15.391416160583496, 0.06086434243835061],
 [2458556.11921799, 15.39814130706787, 0.06614714361829581],
 [2458527.16659628, 15.084998902130128, 0.13455950442294484]]

In [42]:
np.savetxt("2018hna_mags.csv", JD_and_mags, delimiter=",", header='JD, Mag, Mag_Error')